# Wedge Task 2- Owner Sample

The following looks at taking all of GBQ tables and sampling the owners from these tables. 

In [19]:
#imports for the code
import os
from google.cloud import bigquery
from google.oauth2 import service_account
import io
import csv
import random

In [2]:
# Linking to my permissions and GBQ project
service_path = "/Users/kayleeosentowski/Desktop/MSBA/ADA_21/Wedge/"
service_file = 'osentowski-project-3a5d8e60c1a3.json' # authentication information  
gbq_proj_id = 'osentowski-project' # project. 

# using my machine math and json to create my private key
private_key =service_path + service_file

In [3]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [4]:
# establish connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [5]:
#Creating a query to pull only the card numbers from all of my tables that are NOT the value of 3.
# 3 means non-owner and we only want owners
query ='''SELECT DISTINCT(card_no) FROM `osentowski-project.Transaction.transArchive*`
WHERE card_no!=3.0''' 

query_job= client.query(
    query,
    location= "US",
)

In [6]:
#creating an object for the query results
owners = client.query(query,location="US")

In [7]:
# building a list of owners
owner_list = []
for owner in owners:
    owner_list.append(owner[0])

In [8]:
#Total number of owners
len(owner_list)

27207

In [9]:
#Using random to pull a sample of 500 owners from the list
owner_sample = random.sample(owner_list,400)

In [10]:
# Using a SQL statement to pull all info, from all tables where the card number is not 3 
owner_trans = '''SELECT * FROM `osentowski-project.Transaction.transArchive*`
WHERE card_no IN ('''

In [11]:
# stringing together all columns (trans) into the str of owner numbers
for owner in owner_sample:
    owner_trans += str(owner)+','

In [12]:
owner_trans = owner_trans[:-1]+')'

In [13]:
random_trans = client.query(owner_trans,location="US")

In [14]:
for trans in random_trans:
    print('\t'.join([str(x) for x in trans]))
    break

2014-09-04 20:43:47+00:00	7.0	84.0	40.0	0000000000179	CITRUS Lemon Organic	I	 	 	2.0	0.3	1.0	0.0	1.99	0.6	1.99	0.0	0.0	0.0	1.0	0.0	0.0	0.0	1.0	0.0	0.0	6.5697267	0.3	0.0	0.0	0.0	0.0	0.0	None	None	0.0	0.0	0.0	None	0.0	None	0.0	1.0	None	0.0	48419.0	1.0	0.0	0.0	20.0


In [15]:
#Listing all headers to clean up txt file we are writing out
headers = ["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status",
           "department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp",
           "wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty",
           "volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus",
           "charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id",
           "trans_id"]

In [16]:
#Writing out text file called "owner_records"
with open ("owner_records.txt","w") as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in random_trans:
        outfile.write(",".join([str(item) for item in row])+ "\n")

Huge shout out to the lab class, Austin, Jack, and Clay for help with this code!